In [1]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pulp

# 데이터 초기화 부분
# ----------------------------------------------
# i의 개수, 즉 수요지의 개수
num_i = ...

# j의 개수, 즉 시설물 입지점의 개수
num_j = ...

# a_i 값의 리스트. 각 수요지 i의 수요량을 나타냅니다.
a = [...]

# b_ij 값의 2차원 리스트. 수요지 i와 시설물 입지점 j 사이의 거리를 나타냅니다.
# 예: b = [[b_11, b_12, ...], [b_21, b_22, ...], ...]
b = ...

# 설치할 시설물의 개수
p = ...
# ----------------------------------------------

# LP 문제 정의. 여기서는 총 거리를 최소화하는 것이 목적입니다.
prob = pulp.LpProblem("Binary_Optimization", pulp.LpMinimize)

# 변수 정의 부분
# ----------------------------------------------
# l_ij는 노드 j의 시설물이 노드 i의 총 수요를 충족하면 1, 아니면 0입니다.
l = pulp.LpVariable.dicts("l", (range(num_i), range(num_j)), 0, 1, pulp.LpBinary)

# k_j는 노드 j에 시설물이 설치되면 1, 아니면 0입니다.
k = pulp.LpVariable.dicts("k", range(num_j), 0, 1, pulp.LpBinary)
# ----------------------------------------------

# 목적함수 정의. 각 수요지와 시설물 입지점 사이의 거리와 해당 수요지의 수요량을 곱한 값을 최소화합니다.
prob += pulp.lpSum([a[i] * b[i][j] * l[i][j] for i in range(num_i) for j in range(num_j)])

# 제약조건 설정 부분
# ----------------------------------------------
# 각 수요지 i는 정확히 하나의 시설물 j에만 할당되어야 합니다.
for i in range(num_i):
    prob += pulp.lpSum([l[i][j] for j in range(num_j)]) == 1

# 설치된 시설물의 총 개수는 p와 동일해야 합니다.
prob += pulp.lpSum([k[j] for j in range(num_j)]) == p

# l_ij는 k_j에 의존적입니다. 즉, k_j가 1일 때만 l_ij는 1이 될 수 있습니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= k[j]

# 각 수요량 a_i는 200과 750 사이의 값을 가져야 합니다.
for i in range(num_i):
    prob += a[i] >= 200
    prob += a[i] <= 750

# 각 거리 b_ij는 최대 1.5km까지만 허용됩니다.
for i in range(num_i):
    for j in range(num_j):
        prob += b[i][j] <= 1.5
# ----------------------------------------------

# 최적화 문제 풀기
prob.solve()

# 결과 출력 부분
# ----------------------------------------------
print(f"Status: {pulp.LpStatus[prob.status]}")

# 각 수요지에 할당된 시설물 입지점 출력
for i in range(num_i):
    for j in range(num_j):
        print(f"l[{i}][{j}] = {l[i][j].varValue}")

# 각 시설물 입지점의 설치 여부 출력
for j in range(num_j):
    print(f"k[{j}] = {k[j].varValue}")
# ----------------------------------------------


TypeError: 'ellipsis' object cannot be interpreted as an integer